In [16]:
# Import geopandas package
import geopandas as gpd
import pandas as pd
import numpy as np
# Read in shapefile and examine data
contiguous_usa = gpd.read_file('data/cb_2018_us_state_20m.shp')
contiguous_usa.head()
from bokeh.io import curdoc, output_notebook
from bokeh.layouts import widgetbox, row, column
from bokeh.plotting import output_notebook, save
from bokeh.models import ColorBar, LogTicker, Slider, HoverTool
from bokeh.resources import CDN
from bokeh.embed import autoload_static
output_notebook()
from datetime import date
today = date.today()

Loading BokehJS ...

In [17]:
colnames = ['DATE', 'VACCINATED', 'NAME']
state_thresholds = pd.read_csv (r'predicted_immunity.csv', names=colnames, header=None)
state_thresholds.head()

,DATE,VACCINATED,NAME
0,2021-05-29,7374301.0,Alabama
1,2021-05-20,1108648.0,Alaska
2,2021-04-27,10934061.0,Arizona
3,2021-05-01,60228699.0,California
4,2021-04-26,8659207.0,Colorado


In [18]:
date_ranges = [(state_thresholds['DATE'] > '03-01-01') & (state_thresholds['DATE'] <= '2021-04-15'),
              (state_thresholds['DATE'] > '2021-04-15') & (state_thresholds['DATE'] <= '2021-05-01'),
              (state_thresholds['DATE'] > '2021-05-01') & (state_thresholds['DATE'] <= '2021-05-15'),
              (state_thresholds['DATE'] > '2021-05-15') & (state_thresholds['DATE'] <= '2021-06-01'),
              (state_thresholds['DATE'] > '2021-06-01') & (state_thresholds['DATE'] <= '2021-06-15'),
              (state_thresholds['DATE'] > '2021-06-15') & (state_thresholds['DATE'] <= '2021-07-01'),
              (state_thresholds['DATE'] > '2021-07-01') & (state_thresholds['DATE'] <= '2021-07-15'),
              (state_thresholds['DATE'] > '2021-07-15') & (state_thresholds['DATE'] <= '2021-08-01'),
              (state_thresholds['DATE'] > '2021-08-01') & (state_thresholds['DATE'] <= '2021-08-15'),
              (state_thresholds['DATE'] > '2021-08-15') & (state_thresholds['DATE'] <= '2021-09-15'),
              (state_thresholds['DATE'] > '2021-09-01') & (state_thresholds['DATE'] <= '2021-09-15'),
              (state_thresholds['DATE'] > '2021-09-15')]

values = ['spring', 'late april', 'early may', 'late may', 'early june', 'late june', 'early july', 'late july', 'early august', 'late august', 'early september', 'fall']

In [19]:
state_thresholds['estimates'] = np.select(date_ranges, values)
state_thresholds['default_rank'] = state_thresholds['DATE'].rank().astype(int)
state_thresholds['VACCINATED'] = state_thresholds['VACCINATED'].astype(float)
state_thresholds['TOTAL_VACCINATED'] = state_thresholds['VACCINATED'].divide(other = 2)
state_thresholds['TOTAL_VACCINATED'] = state_thresholds.apply(lambda x: "{:,}".format(x['TOTAL_VACCINATED']), axis=1)

state_thresholds.head()

,DATE,VACCINATED,NAME,estimates,default_rank,TOTAL_VACCINATED
0,2021-05-29,7374301.0,Alabama,late may,38,"3,687,150.5"
1,2021-05-20,1108648.0,Alaska,late may,34,"554,324.0"
2,2021-04-27,10934061.0,Arizona,late april,10,"5,467,030.5"
3,2021-05-01,60228699.0,California,late april,16,"30,114,349.5"
4,2021-04-26,8659207.0,Colorado,late april,8,"4,329,603.5"


In [20]:
# Merge shapefile with population data
pop_states = contiguous_usa.merge(state_thresholds, left_on = 'NAME', right_on = 'NAME')

# Drop Alaska and Hawaii
pop_states = pop_states.loc[~pop_states['NAME'].isin(['Alaska', 'Hawaii'])]

In [21]:
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, CategoricalColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.palettes import d3
from bokeh.plotting import figure
# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson = pop_states.to_json())

In [22]:
def json_data(selectedYear):
    yr = selectedYear
    df_yr = df[df['year'] == yr]
    merged = gdf.merge(df_yr, left_on = 'country_code', right_on =     'code', how = 'left')
    merged.fillna('No data', inplace = True)
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data

In [23]:
# Define color palettes
palette = brewer['RdYlGn'][10]
palette = palette[::1] # reverse order of colors so higher values have darker colors
# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low= 0, high = 50)
# Define custom tick labels for color bar.
tick_labels = {'1': '5', '6': '10',
 '11':'15', '16':'20',
 '21':'25', '26':'30',
 '31':'35', '36':'40',
 '41':'50+'}
# Create color bar.
color_bar = ColorBar(color_mapper = color_mapper, 
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'horizontal',
                     major_label_overrides = tick_labels)
# Create figure object.
p = figure(title = '', 
           plot_height = 600, plot_width = 950, 
           toolbar_location = 'below')
p.xaxis.visible= False
p.yaxis.visible= False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.toolbar_location = None
p.outline_line_color = None
# Add patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field' :'default_rank',
                                 'transform' : color_mapper},
                   line_color = 'gray', 
                   line_width = 0.50, 
                   fill_alpha = 1)
# Create hover tool
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@NAME'),
                               ('Total Vaccinated', '@TOTAL_VACCINATED'),
                                 ('Herd Immunity Estimated by', '@estimates')]))
# Specify layout
#Display plot
js, tag = autoload_static(p, CDN, r"docs/script.js")

In [26]:
file = open(r'docs/script.js', 'w')
file.write(js)
file.close()

In [28]:
print(tag)


<script src="docs/script.js" id="91e223a8-5376-47fc-b061-50de0b4178f4"></script>
